# 数据处理

## 数据处理 - 人口

FEI_PREF_190807154102.csv

FEI_CITY_190807153913.csv

> PREF 为县级精度  
> CITY 为市町村级精度


首先处理 CITY 市町村级精度

"FEI_CITY_190807153913.csv" 为 1980~2015 每五年一次调查, 

地点可能含合计项目, 如:

    北海道 札幌市
    北海道 札幌市 中央区
    北海道 札幌市 北区
    北海道 札幌市 東区
    北海道 札幌市 白石区
    ...

预计保留到市一级即可

In [95]:
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)   # 每行最大字符
pd.set_option('precision', 3)         # 显示数字精度
pd.set_option('display.max_rows', 20) # 预览时最多显示行数

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms

plt.rcParams['figure.figsize'] = 18, 9
plt.rcParams['axes.unicode_minus'] = False     # 显示数字负号
plt.rcParams['font.sans-serif'] = ['SimHei']   # 显示中文字体
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['savefig.dpi'] = 100
mpl.rcParams['font.size'] = 12
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'medium'
plt.style.use('seaborn-whitegrid')

import seaborn as sns


### 预览

In [96]:
# 日文编码用 Shift-JIS
population_df = pd.read_csv("../人口/FEI_CITY_190807153913.csv", encoding='shift-jis', header=6)

In [97]:
def explore_df(df):
  print('==== row, col ====', df.shape)
  print('==== columns ====')
  print('\t'.join(df.columns))

  print('==== describe ====')
  print(df.describe())
  print('==== info ====')
  print(df.info())
  print('==== sample ====')
  return df.sample(n=5, random_state=100)
  
# explore_df(df)

def find_in_df(df, col, pat):
  ''' 在 col 中 查找命中 pat 的数据
      如果 pat 是 字符串, 视为模糊查找 
      如果 pat 是 list, 或者非字符串型, 视为精确查找 '''
  if isinstance(pat, str):
    return df[df[col].str.contains(pat)]
  if not isinstance(pat, (list, set)):
    pat = [pat]
  return df[df[col].isin(pat)]


In [98]:
population_df.head(10)
# explore_df(population_df)

,調査年 コード,調査年,地域 コード,地域,/項目,A1101_総人口【人】,注釈,A110101_総人口（男）【人】,注釈.1,A110102_総人口（女）【人】,注釈.2
0,2015100000,2015年度,1100,北海道 札幌市,NaN,"1,952,356",NaN,"910,614",NaN,"1,041,742",NaN
1,2015100000,2015年度,1101,北海道 札幌市 中央区,NaN,"237,627",NaN,"107,756",NaN,"129,871",NaN
2,2015100000,2015年度,1102,北海道 札幌市 北区,NaN,"285,321",NaN,"134,534",NaN,"150,787",NaN
3,2015100000,2015年度,1103,北海道 札幌市 東区,NaN,"261,912",NaN,"124,649",NaN,"137,263",NaN
4,2015100000,2015年度,1104,北海道 札幌市 白石区,NaN,"209,584",NaN,"99,020",NaN,"110,564",NaN
5,2015100000,2015年度,1105,北海道 札幌市 豊平区,NaN,"218,652",NaN,"101,482",NaN,"117,170",NaN
6,2015100000,2015年度,1106,北海道 札幌市 南区,NaN,"141,190",NaN,"65,305",NaN,"75,885",NaN
7,2015100000,2015年度,1107,北海道 札幌市 西区,NaN,"213,578",NaN,"98,648",NaN,"114,930",NaN
8,2015100000,2015年度,1108,北海道 札幌市 厚別区,NaN,"127,767",NaN,"58,312",NaN,"69,455",NaN
9,2015100000,2015年度,1109,北海道 札幌市 手稲区,NaN,"140,999",NaN,"66,284",NaN,"74,715",NaN


### 基本清理, 保留有用的column, 去掉市町村下一级

In [99]:
column_map = [
  ('調査年',                        '调查年份'),
  ('地域 コード',                   '地域数字ID'),
  ('地域',                         '地域'),
  ('A1101_総人口【人】',            '人口数量'),
  ('A110101_総人口（男）【人】',     '人口数量男'),
  ('A110102_総人口（女）【人】',     '人口数量女'),
]

population_df = population_df[[line[0] for line in column_map]]
population_df.columns = [line[1] for line in column_map]
population_df.head(5)

,调查年份,地域数字ID,地域,人口数量,人口数量男,人口数量女
0,2015年度,1100,北海道 札幌市,"1,952,356","910,614","1,041,742"
1,2015年度,1101,北海道 札幌市 中央区,"237,627","107,756","129,871"
2,2015年度,1102,北海道 札幌市 北区,"285,321","134,534","150,787"
3,2015年度,1103,北海道 札幌市 東区,"261,912","124,649","137,263"
4,2015年度,1104,北海道 札幌市 白石区,"209,584","99,020","110,564"


In [100]:
# 如果地域细分到市下一级, 会有两个空格分隔, 如 `北海道 札幌市 中央区`, 
# 去掉这些数据

population_df = population_df[population_df.地域.map(lambda s: s.count(' ') <= 1)]
print(population_df.shape)
population_df.head(5)
# 完后 15328 行变成了 13928 行

(13928, 6)


,调查年份,地域数字ID,地域,人口数量,人口数量男,人口数量女
0,2015年度,1100,北海道 札幌市,"1,952,356","910,614","1,041,742"
11,2015年度,1202,北海道 函館市,"265,979","120,376","145,603"
12,2015年度,1203,北海道 小樽市,"121,924","54,985","66,939"
13,2015年度,1204,北海道 旭川市,"339,605","156,402","183,203"
14,2015年度,1205,北海道 室蘭市,"88,564","43,143","45,421"


In [101]:
population_df.tail(5)

,调查年份,地域数字ID,地域,人口数量,人口数量男,人口数量女
15323,1980年度,47361,沖縄県 久米島町,"10,187",***,***
15324,1980年度,47362,沖縄県 八重瀬町,"18,990",***,***
15325,1980年度,47375,沖縄県 多良間村,"1,667",***,***
15326,1980年度,47381,沖縄県 竹富町,"3,376",***,***
15327,1980年度,47382,沖縄県 与那国町,"2,119",***,***


In [102]:
def convert_num(s):
  if s == '***':
    return np.nan
  return int(s.replace(',', ''))

population_df['人口数量'] = population_df['人口数量'].map(convert_num)
population_df['人口数量男'] = population_df['人口数量男'].map(convert_num)
population_df['人口数量女'] = population_df['人口数量女'].map(convert_num)

explore_df(population_df)
# 人口数量男 人口数量女 因为缺失了一些数据, 这两个字段被算做了 float 型

==== row, col ==== (13928, 6)
==== columns ====
调查年份	地域数字ID	地域	人口数量	人口数量男	人口数量女
==== describe ====
          地域数字ID       人口数量      人口数量男      人口数量女
count  13928.000  1.393e+04  6.964e+03  6.964e+03
mean   21145.743  7.159e+04  3.570e+04  3.751e+04
std    14331.417  1.753e+05  8.985e+04  9.327e+04
min     1100.000  0.000e+00  0.000e+00  0.000e+00
25%     8234.000  9.690e+03  4.344e+03  4.659e+03
50%    20403.000  2.639e+04  1.249e+04  1.354e+04
75%    33203.000  6.421e+04  3.180e+04  3.357e+04
max    47382.000  3.725e+06  1.856e+06  1.869e+06
==== info ====
<class 'pandas.core.frame.DataFrame'>
Int64Index: 13928 entries, 0 to 15327
Data columns (total 6 columns):
调查年份      13928 non-null object
地域数字ID    13928 non-null int64
地域        13928 non-null object
人口数量      13928 non-null int64
人口数量男     6964 non-null float64
人口数量女     6964 non-null float64
dtypes: float64(2), int64(2), object(2)
memory usage: 761.7+ KB
None
==== sample ====


,调查年份,地域数字ID,地域,人口数量,人口数量男,人口数量女
6731,2000年度,21210,岐阜県 恵那市,57274,27785.0,29489.0
6234,2000年度,9384,栃木県 塩谷町,14171,6876.0,7295.0
9457,1995年度,44206,大分県 臼杵市,46830,NaN,NaN
2487,2010年度,11242,埼玉県 日高市,57473,28548.0,28925.0
1836,2015年度,46208,鹿児島県 出水市,53758,24984.0,28774.0


In [113]:
# 检查 `人口数量` 是否有缺失
population_df[population_df[['人口数量']].applymap(pd.isna).all(1)]

,调查年份,地域数字ID,地域,人口数量,人口数量男,人口数量女


In [114]:
# 检查 '人口数量男', '人口数量女' 是否有缺失
# 不必管
population_df[population_df[['人口数量男', '人口数量女']].applymap(pd.isna).all(1)]

,调查年份,地域数字ID,地域,人口数量,人口数量男,人口数量女
7664,1995年度,1100,北海道 札幌市,1757025,NaN,NaN
7675,1995年度,1202,北海道 函館市,318308,NaN,NaN
7676,1995年度,1203,北海道 小樽市,157022,NaN,NaN
7677,1995年度,1204,北海道 旭川市,360568,NaN,NaN
7678,1995年度,1205,北海道 室蘭市,109766,NaN,NaN
7679,1995年度,1206,北海道 釧路市,209680,NaN,NaN
7680,1995年度,1207,北海道 帯広市,171715,NaN,NaN
7681,1995年度,1208,北海道 北見市,131544,NaN,NaN
7682,1995年度,1209,北海道 夕張市,17116,NaN,NaN
7683,1995年度,1210,北海道 岩見沢市,97042,NaN,NaN
